# Servidor HTTP - REST api
*autor: André Costa (2019-03-09)* [https://www.linkedin.com/in/a-l-costa]

- Implementação de um servidor HTTP básico para ofereçer uma api REST para acesso aos dados do warehouse
- **AVISO:** ESTA IMPLEMENTAÇÃO É UMA PROVA DE CONCEITO E NÃO DEVE SER UTILIZADA EM PRODUÇÃO

- **END POINTS**:
    - /estabelecimentos/lista : retorna uma lista de todos os estabelecimentos que possuem algum evento de fluxo registrado, expondo o ID (SK) e o nome dos estabelecimentos. Retornado como *text/plain*, mas formatado como *csv*. Duas columas.
    - /estabelecimentos/detalhes/*ID/ : retorna um único registro com informações detalhadas sobre o estabelecimento, incluindo o fluxo médio para cada dia da semana, em cada período. Retornado como *text/plain*, mas formatado como *csv*. O número de colunas é variável.

*OBSERVAÇÃO:* para utilizar, execute todos os passos deste script e mantenha o kernel rodando. acesse como localhost:8008/ pelo host

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import UserDefinedFunction
import pandas as pd
import numpy as np

spark.sql('use geodata')

DataFrame[]

In [2]:
def list_estabelecimentos(opts):
    validConc = spark.sql('select id, nome FROM concorrente WHERE isnull(end_date) \
        and id in (select concorrente_id FROM fluxo group by concorrente_id)')
    data = validConc.toPandas().to_csv(None, index=False)
    return {
        'status': 200,
        'type': 'text/plain',
        'content': data
    }

In [3]:
week_day = {0: 'dom', 1: 'seg', 2: 'ter', 3: 'qua', 4: 'qui', 5: 'sex', 6: 'sab'}
periodo = {0: 'manha', 1: 'tarde', 2: 'noite'}

def name_segments(day,p):
    return '_'.join([week_day[day], periodo[p]])
map_name_segments = UserDefinedFunction(name_segments, StringType())

def estabelecimento_details(opts):
    try:
        c_id = int(opts[0])
        concProfile = spark.sql('select a.id, a.codigo, a.nome, a.endereco, a.faixa_preco, \
            b.nome as bairro, b.populacao, b.dens_demo FROM concorrente as a INNER join bairro as b \
            ON a.id={} and a.bairro_id=b.codigo and isnull(b.end_date)'.format(c_id))
        if not concProfile.count():
            return {
                'status': 404,
                'type': 'text/plain',
                'content': u'Registro não encontrado.'
            }

        profile_data = concProfile.toPandas()
        H1 = list(profile_data)
        R1 = profile_data.values[0]
        R1[2] = '\"{}\"'.format(R1[2])
        R1[3] = '\"{}\"'.format(R1[3])

        fluxo = spark.sql('select AVG(a.ocorrencias) as fluxo_medio, b.dia_semana, b.periodo \
            FROM fluxo as a INNER join calendario as b \
            ON a.calendario_id=b.id WHERE concorrente_id={} group by b.dia_semana, b.periodo \
            order by b.dia_semana, b.periodo'.format(c_id))

        fluxo = fluxo.withColumn('seg', map_name_segments(fluxo.dia_semana, fluxo.periodo)) \
            .drop('dia_semana', 'periodo')

        fluxo_data = fluxo.toPandas().values.transpose()
        H2 = fluxo_data[1].ravel()
        R2 = fluxo_data[0].ravel()

        H = ','.join(np.hstack((H1, H2)))
        R = ','.join([str(x) for x in np.hstack((R1, R2))])
        data = '\n'.join([H, R])

        return {
            'status': 200,
            'type': 'text/plain',
            'content': data
        }
    
    except:
        return {
            'status': 500,
            'type': 'text/plain',
            'content': u'Não foi possível obter os detalhes do estabelecimento. \
                verifique sua requisição.'
        }

In [4]:
handlers = {
    'lista': list_estabelecimentos,
    'detalhes': estabelecimento_details
}

def handle_estabelecimentos(opts):
    try:
        return handlers[opts[0]](opts[1:])
    except:
        return {
            'status': 500, 
            'content': u'Operação para "estabelecimentos" inválida. \
                Verifique sua requisição.', 
            'type': 'text/plain'
        }

def split_path(path):
    path_list = [x for x in path.split('/') if x!='']
    path_list.append('')
    return path_list

entry_handlers = {
    'estabelecimentos': handle_estabelecimentos
}

In [5]:
import time
from http.server import BaseHTTPRequestHandler, HTTPServer

HOST_NAME = 'pyspark-server'
PORT_NUMBER = 8008


class MyHandler(BaseHTTPRequestHandler):
    def do_HEAD(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/plain')
        self.end_headers()

    def do_GET(self):
        opts = split_path(self.path)
        try:
            response = entry_handlers[opts[0]](opts[1:])
        except:
            response = {
                'status': 404, 
                'content': u'Requisição inválida: {}'.format(self.path),
                'type': 'text/plain'
            }
        
        self.respond(response)

    def handle_http(self, status_code, content_type, content):
        self.send_response(status_code)
        self.send_header('Content-type', content_type)
        self.end_headers()
        return bytes(content, 'UTF-8')

    def respond(self, response):
        data = self.handle_http(response['status'], response['type'], response['content'])
        self.wfile.write(data)

if __name__ == '__main__':
    server_class = HTTPServer
    httpd = server_class((HOST_NAME, PORT_NUMBER), MyHandler)
    print(time.asctime(), 'Server Starts - %s:%s' % (HOST_NAME, PORT_NUMBER))
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    print(time.asctime(), 'Server Stops - %s:%s' % (HOST_NAME, PORT_NUMBER))

Mon Mar 11 22:20:34 2019 Server Starts - pyspark-server:8008


172.17.0.1 - - [11/Mar/2019 22:20:58] "GET /estabelecimentos/lista HTTP/1.1" 200 -


Mon Mar 11 22:21:23 2019 Server Stops - pyspark-server:8008
